# SO, SO2, PPD: comunicação e sincronização de *threads*

Hélio - DC/UFSCar - 2023

# Sincronização de *threads*

*Threads* compartilham o espaço de endereçamento do processo ao qual estão associadas. Isso facilita as comunicações entre elas, uma vez que quaisquer estruturas manipuladas de maneira particionada podem ser posicionadas em áreas de memória comuns, acessíveis por todas as *threads*.

Por outro lado, a consistência dos dados compartilhados pode depender de certos acessos ocorrerem de **maneira exclusiva** e é claro que essa sincronização não seria eficiente se fosse implementada com algum tipo de ***busy waiting***, salvo em bloqueios de curtíssima duração.

```
while(recurso_ocupado())
  fica_aqui_à_toa_gastando_fatia_de_tempo_até_outra_tarefa_liberá-lo();
```

Assim, a ideia é ter mecanismos de sincronização e exclusão mútua que possibilitem que as tarefas sejam automaticamente suspensas quando condições necessárias não são satisfeitas.

Desta forma, a tarefa que aguarda uma condição pode ser tirada de execução e outra tarefa pode ser selecionada para ocupar o processador que se tornou disponível agora.

O primeiro passo para isso é ter uma biblioteca padronizada que ofereça esses serviços. No cado do uso de *threads* com a biblioteca ***pthreads***, há uma série de estruturas de sincronização, como pode ser visto em [pthread.h](https://man7.org/linux/man-pages/man0/pthread.h.0p.html).

É útil para a aplicação usar bibliotecas padronizadas, já que isso aumenta a portabilidade dos programas.

* No caso específico do sistema Linux, por razões de eficiência, as implementações dessas bibliotecas procuram fazer todas as operações possíveis no "espaço de usuário" (*user space*), evitando o custo dos mecanismos de chamada de sistema. Isto é importante, já que a eficiência dos mecanismos de sincronização é fundamental para o bom desempenho das aplicações.

* Quando o apoio do SO (Linux) é necessário para as sincronizações, a chamada [futex](https://man7.org/linux/man-pages/man2/futex.2.html) pode ser usada, por exemplo, tanto para bloquear (***FUTEX_WAIT***), quanto para acordar (***FUTEX_WAKE***) tarefas.

Quanto vai bloquear uma tarefa, o SO primeiro ajusta e salva informações de contexto da tarefa que emite a chamada. O descritor desta tarefa é então "desligado" da lista de tarefas prontas e inserido na fila das tarefas que aguardam que essa condição seja satisfeita. Tipicamente, a tarefa é inserida na fila de tarefas associada ao semáforo ou mutex em questão (ou futex, no Linux). O SO segue selecionando outra tarefa pronta para execução e restaurando seu contexto para que ela volte a ser executada.

Pensando que estamos estudando a lógica dos mecanismos de sincronização e que queremos criar programas portáteis, sigamos usando as APIs POSIX e os mecanismos providos para isto!

Faz sentido?


# RWlocks: bloqueios para leitura ou escrita

A API para bloqueios de leitura e escrita (***rwlocks***) permite fazer o bloqueio seletivo de leitores e escritores que desejam manipular alguma estrutura compartilhada.

Bloqueios de leitura impedem apenas o acesso de escritores à estrutura mas não impedem que outros leitores realizem o mesmo bloqueio. Já bloqueios de escrita impedem o acesso de outros escritores e também de leitores.

Deste modo, um bloqueio de leitura só é realizado se não há um bloqueio de escrita ativo. Por outro lado, pode haver vários bloqueios de leitura sobre a mesma estrutura e, para que o acesso de escrita seja liberabo, é preciso que todos esses bloqueios sejam desfeitos.

Com as chamadas padrão, ***rwlock_rdlock*** e ***rwlock_wrlock***, a tarefa que as emite só prossegue execução caso o bloqueio solicitado seja realizado. Em caso contrário, a tarefa é suspensa.

Contudo, omo se vê nas funções disponíveis pela API, também é possível emitir chamadas não bloqueantes, com as opções ***tryrdlock*** e ***trywrlock***. Já as opções ***timedrdlock*** e ***timedwrlock*** usam um parâmetro a mais, que indica o tempo máximo de bloqueio da tarefa em caso de a operação solicitada não ser possível.

Assim, no uso das chamadas ***\_try_*** e ***\_timed_***, é preciso avaliar o valor de retorno para saber se o bloqueio solicitado foi realizado ou não.

* int [pthread_rwlock_init](https://man7.org/linux/man-pages/man3/pthread_rwlock_init.3p.html) (pthread_rwlock_t *restrict rwlock, const pthread_rwlockattr_t *restrict attr);

* int [pthread_rwlock_rdlock](https://man7.org/linux/man-pages/man3/pthread_rwlock_rdlock.3p.html) (pthread_rwlock_t *rwlock);
* int [pthread_rwlock_tryrdlock](https://man7.org/linux/man-pages/man3/pthread_rwlock_tryrdlock.3p.html) (pthread_rwlock_t *rwlock);
* int [pthread_rwlock_timedrdlock](https://man7.org/linux/man-pages/man3/pthread_rwlock_timedrdlock.3p.html) (pthread_rwlock_t *restrict rwlock, const struct timespec *restrict abs_timeout);

* int [pthread_rwlock_wrlock](https://man7.org/linux/man-pages/man3/pthread_rwlock_wrlock.3p.html) (pthread_rwlock_t *rwlock);
* int [pthread_rwlock_trywrlock](https://man7.org/linux/man-pages/man3/pthread_rwlock_trywrlock.3p.html) (pthread_rwlock_t *rwlock);
* int [pthread_rwlock_timedwrlock](https://man7.org/linux/man-pages/man3/pthread_rwlock_timedwrlock.3p.html) (pthread_rwlock_t *restrict rwlock, const struct timespec *restrict abs_timeout);

* int [pthread_rwlock_unlock](https://man7.org/linux/man-pages/man3/pthread_rwlock_unlock.3p.html) (pthread_rwlock_t *rwlock);

* int [pthread_rwlock_destroy](https://man7.org/linux/man-pages/man3/pthread_rwlock_destroy.3p.html) (pthread_rwlock_t *rwlock);

<br>

Um bloqueio efetivado, seja ele de leitura ou escrita, é liberado com a chamada ***pthread_rwlock_unlock***(). Quando se trata de um bloqueio de leitura, a implementação verifica o número de bloqueios ativos e o decrementa. Apenas caso este seja o último bloqueio de leitura é que o acesso é liberado para escrita.

<br>

De maneira geral, vê-se que esta API soliciona o uso de acessos de leitura e escrita. Será, contudo, que isso impede a ocorrência de ***starvation***? Ou seja, será que simplesmente usar os bloqueios apropriados de leitura e de escrita garante que um escritor não será indefinidamente preterido,  porque sempre há algum leitor ativo?

<br>

Vejamos um exemplo de código a seguir.


In [ ]:
%%writefile rwlock.c

/*
** UFSCar / DC
** Prof. Hélio Crestana Guardia
*/

#include <stdio.h>
#include <unistd.h>
#include <stdlib.h>
#include <pthread.h>

#define NT 4

pthread_rwlock_t _rwlock;

// criacao e inicializacao estatica
pthread_mutex_t _lmutex = PTHREAD_MUTEX_INITIALIZER;
pthread_mutex_t _emutex = PTHREAD_MUTEX_INITIALIZER;


int _val = -1,      // variável compartilhada
	_leitores = 0,    // número de leitores ativos
	_escritores = 0;  // número de escritores ativos


void *
leitor(void *param)
{
	long int id = (long int)param;
	srand(time(NULL));

	do {
		sleep(random()%5); // dorme antes de tentar obter bloqueio para leitura

		// tenta obter bloqueio para leitura; não deve impedir outros leitores
		// if (pthread_rwlock_tryrdlock(&_rwlock)==0) {
  	if (pthread_rwlock_rdlock(&_rwlock)==0) {

			// mutex para manipular o contador...
			pthread_mutex_lock(&_lmutex);
				_leitores = _leitores+1; // incrementa número de leitores
			pthread_mutex_unlock(&_lmutex);

			printf("%d lendo (%d). %d leitor(es) ativo(s)\n", (int)id, _val, _leitores);

			sleep(random()%5); // passa um tempo aleatório simulando leitura e uso do dado

			// printf("Leitor %d terminou de ler\n", id);

			pthread_mutex_lock(&_lmutex);
				_leitores = _leitores-1; // decrementa número de leitores
			pthread_mutex_unlock(&_lmutex);

			printf("leitor %d terminando. %d leitor(es) ativo(s)\n", (int)id, _leitores);
			// libera bloqueio para leitura e escrita
			// será que sistema mantém contador de bloqueios de leitura?
			// recurso só deve ser liberado para escrita se contador = 0...
			pthread_rwlock_unlock(&_rwlock);
		}
	} while (1);
}

void *
escritor(void *param)
{
	long int id = (long int)param;
	srand(time(NULL));

	do {
		sleep(random()%5); // dorme antes de tentar obter bloqueio para escrita

		// tenta obter bloqueio para escrita, o que também impede leitores
		// if (pthread_rwlock_trywrlock(&_rwlock)==0)
		if (pthread_rwlock_wrlock(&_rwlock)==0) {

			// mutex para manipular o contador. Se wrlock funcionar, será no máximo 1...
			pthread_mutex_lock(&_emutex);
				_escritores = _escritores+1; // incrementa número de escritores
			pthread_mutex_unlock(&_emutex);

			_val= id; //escreve id em _val

			printf("%d escrevendo... %d escritor ativo\n", (int)id, _escritores);

			sleep(random()%5); // simula duração do bloqueio

			// printf("%d terminou de escrever\n", id);

			// mutex para manipular o contador...
			pthread_mutex_lock(&_emutex);
				_escritores = _escritores-1; // decrementa número de escritores
			pthread_mutex_unlock(&_emutex);

			// libera bloqueio
			pthread_rwlock_unlock(&_rwlock);
		}
	} while (1);
}

int
main(void)
{
  long int i;
	pthread_t leitores[NT];
	pthread_t escritores[NT];

	// inicia um rwlock para ser usado tanto pelo escritor quanto pelo leitor
	pthread_rwlock_init(&_rwlock,NULL);

	for (i = 0; i < NT; i++) {
		//Crias as threads Leitoras
		pthread_create(&leitores[i], NULL, leitor, (void *)i);

		//Cria as threads Escritoras
		pthread_create(&escritores[i], NULL, escritor, (void *)i);
	}

	// espera fim das threads (pthread_exit()) / o que não irá acontecer aqui...
	for (i = 0; i < NT; i++) {
		pthread_join(leitores[i], NULL);
		pthread_join(escritores[i], NULL);
	}

	// remove mutex
	pthread_mutex_destroy (&_lmutex);
	pthread_mutex_destroy (&_emutex);

	// remove rwlock
	pthread_rwlock_destroy(&_rwlock);

	return 0;
}


Writing rwlock.c


In [ ]:
! if [ ! rwlock -nt rwlock.c ]; then gcc -Wall rwlock.c -o rwlock -pthread ; fi
! ./rwlock

^C


# Condition Variables

Como vimos, ***mutexes*** provêm sincronização baseada no valor de uma estrutura binária, que pode estar liberada ou bloqueada. Já ***semáforos***, basicamente, determinam a validade de uma condição em função da possibilidade de decrementar o valor de um contador sem que ele fique negativo.

Será que essas 2 estruturas podem representar todas as condições de sincronização em tarefas?

Aparentemente, não, e é para complementar as funcionalidades dos mecanismos de sincronização que inventaram as ***condition variables*** (variáveis de condição).

<br>

A API de *condition variables* oferece primitivas que permitem a uma aplicação realizar um bloqueio quando alguma condição necessária não estiver satisfeita. De maneira correspondente, há também primitivas para desbloquear tarefa(s) bloqueada(s) anteriormente.

Vale ressaltar que trata-se de **alguma condição**, ou seja, qualquer condição definida pela lógica da aplicação!

Associada à estrutura de uma *condition variable*, há **apenas uma fila de tarefas bloqueadas**. **Não** há um contador, ou algum mecanismo para garantir a exclusão mútua na consulta ou alteração do valor de algum contador.

Vejamos as primitivas desta API:

* int [pthread_cond_init](https://man7.org/linux/man-pages/man3/pthread_cond_init.3p.html) (pthread_cond_t *cond, pthread_condattr_t *cond_attr);
* pthread_cond_t cond = PTHREAD_COND_INITIALIZER; /* outra forma de iniciação */
* int [pthread_cond_signal](https://man7.org/linux/man-pages/man3/pthread_cond_signal.3p.html) (pthread_cond_t *cond);
* int [pthread_cond_broadcast](https://man7.org/linux/man-pages/man3/pthread_cond_broadcast.3p.html) (pthread_cond_t *cond);
* int [pthread_cond_wait](https://man7.org/linux/man-pages/man3/pthread_cond_wait.3p.html) (pthread_cond_t *cond, pthread_mutex_t *mutex);
* int [pthread_cond_timedwait](https://man7.org/linux/man-pages/man3/pthread_cond_timedwait.3p.html) (pthread_cond_t *cond, pthread_mutex_t *mutex, const struct timespec *abstime);
* int [pthread_cond_destroy](https://man7.org/linux/man-pages/man3/pthread_cond_destroy.3p.html) (pthread_cond_t *cond);

<br>

Como disse, a condição que que deve ser satisfeita é **definida pela aplicação**. Comumente, essa condição é **determinada por alguma expressão envolvendo alguma variável**.

Se a análise da condição envolve alguma variável que pode ser lida ou alterada por várias *threads* de uma aplicação, é claro que o acesso a essa variável, ou estrutura, precisa ser **feito com exclusão mútua**. Deste modo, deve haver um ***mutex*** associado à análise das expressões que testam a condição. Cabe à aplicação, contudo, fazer uso correto deste *mutex*.

<br>

Tomemos como exeplo uma aplicação em que múltiplas *threads* precisam tomar uma decisão em função do valor de uma variável.

Sem variáveis de condição, *threads* devem fazer uma varredura constante para verificar se condição está satisfeita. Antes de testar a condição, cada *thread* deve bloquear um *mutex*, já que se outra delas estiver alterando o valor desta variáel, a leitura pode resultar num valor inconsistente (*half writen*).

<br>

Mas o que fazer se a condição não estiver satisfeita?

```
do {
  pthread_mutex_locK (&mut);

  if (func( var ) == 0) {    /* func() é uma expressão sobre var: 0==sucesso */
    // libera o mutex e segue!
    pthread_mutex_unlok (&mut);
    break;
  }
  // else segue no loop em busy wait!
while(1);
```

Neste caso, ao invés de ficar testando continuamente, gastando tempo, poderia ser mais útil direcionar o uso do processador para outra tarefa, e só voltar a testar quando a condição for satisfeita.

```
  pthread_mutex_locK (&mut);

  if (func (var) == 0) {
    // libera o mutex e segue!
    pthread_mutex_unlok ( &mut );      // ------ liberação do mutex

  } else {
      // tarefa se auto-bloqueia na fila da cond var, com o mutex fechado (*)
      pthread_cond_wait ( &cond, &mut);

      // Só vai chegar nesta linha do código quando a condição
      // estiver satisfeita e outra thread acordar esta

      // já pode liberar o mutex agora
      ptread_mutex_unlok ( &mut );     // -------- liberação do mutex
  }
```
(\*) Essa é uma condição estranha, mas é assim que funciona com *cond vars*; veremos...

<br>

Usando *condition variable*, a *thread* neste caso se auto-bloqueou na fila associada a esta estrutura. É claro, então, que caberá a outra *thread* do processo verificar se a condição foi satisfeita e, então desbloquear a(s) *thread*(s) bloqueada(s).

A chamada ***pthread_cond_signal ( &cond )*** serve para que uma *thread* acorde outra bloqueada na fila da *condition variable*. Já ***pthread_cond_broadcast ( &cond )*** serve para acordar **todas** as *threads* que estiverem ali bloqueadas.

<br>

Neste exemplo, é importante atentarmos para o uso do ***mutex*** associado às manipulações desta *condition variable*. Ele foi bloqueado antes da análise da condição. Caso a condição esteja satisfeita, o *mutex* é liberado.

Caso contrário, o *mutex* é **mantido fechado** e uma refrência a ele é passada na operação de bloqueio (pthread_cond_wait). Vai caber à implementação da operação *pthread_cond_wait* **liberar** esse mutex antes de suspender a tarefa. Por outro lado, quando esta tarefa for desbloqueada por outra, este mutex será **novamente bloqueado** automaticamente pela implementação.

<br>

Vejamos mais detalhes sobre as primitivas da API de *condition variables*.


**pthread_cond_wait**()

* Bloqueia a thread que executa a chamada até que a condição especificada seja sinalizada.
* Deve ser executada enquanto o mutex está **bloqueado**.
* Mutex associado é automaticamente liberado enquanto a condição testada não está satisfeita.
* Quando sinal é recebido e thread é desbloqueada (acordada), mutex é bloqueado automaticamente para uso pela thread.
* Programador deve incluir chamada explícita para desbloquear mutex ao final da seção crítica (depois de chamar pthread_cond_signal())

**pthread_cond_signal**()

* Sinaliza (ou acorda) outra thread que estiver esperando pela variável condicional.
* Deve ser executada enquanto o mutex está bloqueado e deve liberar o mutex para que a função pthread_cond_wait seja completada.

**pthread_cond_broadcast**()

* Deve ser usada (ao invés de pthread_cond_signal) para liberar mais de uma threads bloqueadas à espera de variável de condição.

<br>

## Exemplo *condition variable*

```
Main thread
   Declara e inicializa variáveis globais que requerem acesso sincronizado (ex. count)
   Declara e inicializa uma condition variable
   Declara e inicializa um mutex
   Cria threads A e B
   Join / continua

Thread A
  Executa até o ponto em que uma condição deva ser satisfeita (Ex. count com valor x)
  Bloqueia mutex associado à conditional variable e verifica valor da variável global (count)
    Executa pthread_cond_wait() para esperar de maneira bloqueante até que condição seja sinalizada pela thread B.
    // Chamada a pthread_cond_wait automaticamente desbloqueia o mutex de maneira atômica, que agora pode ser bloqueado por B.
    Quando receber a sinalização, é acordada. Mutex é bloqueado de maneira atômica.
     Faz o que tem que fazer quando a condição está satisfeita
  Explicitamente libera mutex (phtread_mutex_unlock).
  Continua

Thread B
  Executa
  Bloqueia mutex
    Altera o valor da variável global monitorada pela thread A
    Verifica valor da variável global esperada pela thread A. Se valor for satisfeito, sinaliza (pthread_cond_signal) thread A
  Libera o mutex
  Continua
```

<br>

Vejamos agora um exemplo de código do tipo produtor/consumidor, com sincronização usando *condition variables*.

<br>


In [ ]:
%%writefile pc-condvar.c

/*
 * UFSCar / DC
 * Prof. Hélio Crestana Guardia
 */

/*
 * Programa : produtor / consumidor usando condition variable
              para controlar o acesso ao vetor de dados.

        Consumidor só é ativado quando buffer está cheio.
        Produtor só é ativado quando buffer está vazio.
        Inicialmente, produtor e consumidores podem atuar, mas consumidores
         são bloqueados já que não há dado produzido ainda.
*/

#include <pthread.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>

#define LEN 128
#define MAX_ITEMS 4
#define N_CONS    4

// conditional variables
pthread_cond_t cond_produtor, cond_consumidor;

// mutex
pthread_mutex_t _mutex;

int _fim=0;

// buffer compartilhado
int _buffer[MAX_ITEMS];
int _in, _out;
int _numitems;

void
insere_item(int item)
{
  _buffer[_in]=item;
  _in=(_in+1) % MAX_ITEMS;

  printf("Inseriu %d (%d)\n",item,_numitems);
}
int
remove_item()
{
  int item;

  item=_buffer[_out];
  _out=(_out+1) % MAX_ITEMS;

  return item;
}

void
processa_item(int id, int item)
{
  printf("%d processando %d (%d)\n",id,item,_numitems);
  sleep(rand()%10);
}

int
produz_item()
{
  sleep(rand()%3);
  return(rand()%10);
}

void *produtor(void *arg)
{
  int item;

  do {

    item=produz_item();

    // bloqueia mutex
    pthread_mutex_lock(&_mutex);

    // Produtor só é reativado quando buffer estiver completamente vazio
    // verificação da condição deve ser feita antes da chamada, ou não, de cond_wait
    // cond_wait automaticamente desbloqueia o mutex indicado
    if (_numitems == MAX_ITEMS) { // buffer está cheio

      // sinaliza liberação da condição para consumidor
      printf("Produtor vai liberar consumidores...%d itens\n",_numitems);

      // The pthread_cond_signal() function shall unblock at least one of the threads that are blocked on the specified condition variable cond.
      // The pthread_cond_broadcast() function shall unblock all threads currently blocked on the specified condition variable cond.
      // pthread_cond_signal(&cond_consumidor);
      pthread_cond_broadcast(&cond_consumidor);

      // bloqueia-se à espera do consumidor esvaziar a fila
      printf("Produtor vai bloquear...%d itens\n",_numitems);
      pthread_cond_wait(&cond_produtor, &_mutex);

      // se chegou aqui depois de ter sido bloqueado (porque a condição não
      // estava satisfeita), é porque thread consumidor liberou (cond_signal) a
      // variável condicional
      // Mutex é automaticamente rebloqueado
      // else
      // não precisa pois condição está satisfeita e mutex já está bloqueado
    }
    insere_item(item);
    _numitems++;

    // desbloqueia mutex
    pthread_mutex_unlock(&_mutex);

  } while (1);

  pthread_exit(NULL);
}

void *consumidor(void *arg)
{
  int item;
  long int id = (long int)arg;

  do {
    // bloqueia mutex
    pthread_mutex_lock(&_mutex);

    // Consumidor só é reativado quando buffer estiver completamente cheio
    // verificação da condição deve ser feita antes da chamada de cond_wait
    // cond_wait automaticamente desbloqueia o mutex indicado
    if (_numitems == 0) { // buffer está vazio

      // bloqueia-se à espera do produtor encher a fila
      printf("Consumidor %d vai bloquear...%d itens\n",(int)id,_numitems);
      pthread_cond_wait(&cond_consumidor, &_mutex);
      // se chegou aqui depois de ter sido bloqueado (porque a condição não
      // estava satisfeita) é porque outra thread liberou (cond_signal) a
      // variável condicional
      // Mutex é automaticamente rebloqueado
    }
    item = remove_item();
    _numitems--;

    if (_numitems==0) {
      // sinaliza liberação da condição para produtor
      printf("Consumidor %d vai liberar produtor...%d itens\n",(int)id,_numitems);
      pthread_cond_signal(&cond_produtor);
    }

    // desbloqueia mutex
    pthread_mutex_unlock(&_mutex);

    processa_item(id,item);

  } while(1);

  pthread_exit(NULL);
}


int
main (int argc, char *argv[])
{
  pthread_t prod_t, cons_t[N_CONS];
  int result;
  long int t;
  char err_msg[LEN];

  _in=0;
  _out=0;
  _numitems = 0;

  srandom(getpid());

  // inicia variáveis de condição
  pthread_cond_init(&cond_consumidor, NULL);
  pthread_cond_init(&cond_produtor, NULL);

  // inicia mutex usado para controlar manipulação dos ponteiros do buffer
  pthread_mutex_init(&_mutex, NULL);

  // cria produtor
  result = pthread_create(&prod_t, NULL, produtor, NULL);
  if (result) {
    strerror_r(result,err_msg,LEN);
    printf("Erro criando produtor: %s\n",err_msg);
    exit(0);
  }
  // cria consumidores
  for(t=0; t < N_CONS; t++) {
    result = pthread_create(&(cons_t[t]), NULL, consumidor, (void *)t);
    if (result) {
      strerror_r(result,err_msg,LEN);
      printf("Erro criando consumidor %d: %s\n",(int)t,err_msg);
      exit(0);
    }
  }
  // criar condição para fim de programa (_fim=1)?

  // espera threads terminarem (ou não?)
  result = pthread_join(prod_t, NULL);
  if (result) {
    strerror_r(result,err_msg,LEN);
    fprintf(stderr,"Erro em pthread_join: %s\n",err_msg);
  }
  for(t=0;t<N_CONS;t++) {
    result = pthread_join(cons_t[t], NULL);
    if (result) {
      strerror_r(result,err_msg,LEN);
      fprintf(stderr,"Erro em pthread_join: %s\n",err_msg);
    }
  }

  pthread_mutex_destroy(&_mutex);

  pthread_cond_destroy(&cond_consumidor);
  pthread_cond_destroy(&cond_produtor);

  return 0;
}


Overwriting pc-condvar.c


In [ ]:
! if [ ! pc-condvar -nt pc-condvar.c ]; then gcc -Wall pc-condvar.c -o pc-condvar -pthread ; fi
! ./pc-condvar

Consumidor 0 vai bloquear...0 itens
Consumidor 1 vai bloquear...0 itens
Consumidor 2 vai bloquear...0 itens
Consumidor 3 vai bloquear...0 itens
Inseriu 2 (0)
Inseriu 8 (1)
Inseriu 7 (2)
Inseriu 9 (3)
Produtor vai liberar consumidores...4 itens
Produtor vai bloquear...4 itens
0 processando 2 (3)
1 processando 8 (2)
2 processando 7 (1)
Consumidor 3 vai liberar produtor...0 itens
3 processando 9 (0)
Inseriu 6 (0)
Consumidor 2 vai liberar produtor...0 itens
2 processando 6 (0)
Inseriu 2 (0)
Inseriu 0 (1)
^C


Como se vê no exemplo, a condição de verificação é responsabildade da própria aplicação. Trata-se de solução flexível mas cabe a ela garantir o funcionamento correto.

# SPINs

***SPINs*** são outra estrutura para comunicação e sincronização de *threads*.

Basicamente, *spins* oferecem o mesmo serviço de sincronização lógica para garantia de exclusão mútua provido pelos ***mutexes***. Assim, até as primitivas são parecidas, como se vê a seguir:

* int [pthread_spin_destroy](https://man7.org/linux/man-pages/man3/pthread_spin_destroy.3.html) (pthread_spinlock_t *lock);
* int [pthread_spin_init](https://man7.org/linux/man-pages/man3/pthread_spin_init.3.html) (pthread_spinlock_t *lock, int pshared);
* int [pthread_spin_lock](https://man7.org/linux/man-pages/man3/pthread_spin_lock.3.html) (pthread_spinlock_t *lock);
* int [pthread_spin_trylock](https://man7.org/linux/man-pages/man3/pthread_spin_trylock.3.html) (pthread_spinlock_t *lock);
* int [pthread_spin_unlock](https://man7.org/linux/man-pages/man3/pthread_spin_unlock.3.html) (pthread_spinlock_t *lock);

<br>

O que muda, então?

A resposta é a **forma de bloqueio**. Ao **invés de suspender** a execução da tarefa caso o *spin* (representando uma região crítica) esteja bloqueado, a função fica "girando no lugar" (***spinning*** :-).

Mas para que serve esse mecanismo, então, cujo comportamento é equivalente a fazer um ***busy wait***?

Primeiro, é claro que esse tipo de mecanismo, com espera ociosa, só é interessante para situações em que os **bloqueios são de curtíssima duração**. Nesses casos, é mais eficiente esperar um pouquinho do que realizar uma troca de contexto, quer envolve salvar o estado da execução da tarefa atual, selecionar outra, restaurar seu contexto e colocá-la em execução.

Uma vantagem do uso de *spins* em relaçõa a *busy waiting* na própria tarefa é que as primitivas de *spin* que fazem as verificações da variável de controle associada são executadas de maneira **atômica e segura**.

Faz sentido?

<br>

Algumas informações obtidas a partir do manual das funções de *spin*:

```
	int pthread_spin_lock(pthread_spinlock_t *lock);
	int pthread_spin_trylock(pthread_spinlock_t *lock);

	The pthread_spin_lock() function shall lock the spin lock referenced by lock. The calling thread shall acquire the lock
	if it is not held  by  another  thread.
	Otherwise, the thread shall spin (that is, shall not return from the pthread_spin_lock() call) until
	the lock becomes available.

	The pthread_spin_trylock() function shall lock the spin lock referenced by lock if it is not held by any thread.
	Otherwise, the function shall fail.

	int pthread_spin_init(pthread_spinlock_t *lock, int pshared);
	int pthread_spin_destroy(pthread_spinlock_t *lock);

	The pthread_spin_init() function shall allocate any resources required to use the spin lock referenced by lock
	and initialize the lock to an unlocked state.

	The pthread_spin_destroy() function shall destroy the spin lock referenced by lock and release any resources
	used by the lock.

	int pthread_spin_unlock(pthread_spinlock_t *lock);

	The pthread_spin_unlock() function shall release the spin lock referenced by lock which was locked via the
	pthread_spin_lock() or pthread_spin_trylock() functions.

	If there are threads spinning on the lock when pthread_spin_unlock() is called, the lock becomes available
	and an unspecified spinning thread shall acquire the lock.
```



Vejamos um exemplo de programa com uso de *spins* para garantir exclusão mútua.

Neste caso, a sincronização é usada apenas para garantir que os incrementos à variável compartilhada ocorram de maneira atômica. Assim, trata-se de um bloqueio de curtíssima duração, pelo tempo de ler o valor da variável, somar a ela o valor do identificador da *thread* e salvar o valor novamente na mesma posição de memória.

Em termos de desempenho, seria muito custoso ter uma troca de contexto se, por acaso, uma *thread* encontrasse o *spin* bloqueado, certo?! Afinal, quantas instruções será que há numa troca de contexto?
.


In [ ]:
%%writefile spin.c

/*
 * UFSCar / DC
 * Prof. Hélio Crestana Guardia
 */

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <stdlib.h>
#include <unistd.h>
#include <pthread.h>

#define N_THREADS 3
#define TTL			3
#define LEN 128

// declaracao do spinlock deve ser global
pthread_spinlock_t _slock;

int _result = 0;

void *
soma(void *arg)
{
	int i;
	long int num = (long int)arg;

	printf("Thread %d pronta\n",(int)num);
	// sleep(1);

	for (i=0; i < TTL; i++) {
		printf("Th %d esperando...\n",(int)num); fflush(stdout);

   	pthread_spin_lock(&_slock);

		// printf("Th: %d+%d\n",_result,(int)th_num); fflush(stdout);
		printf("Th: %d na secao critica\n",(int)num); fflush(stdout);

		_result = _result + num;

		// sleep(1);

		printf("Th: %d liberando secao critica...\n",(int)num); fflush(stdout);

		pthread_spin_unlock(&_slock);

		// pthread_yield();
		sleep(1);
	}
	printf("Soma = %d\n",_result); fflush(stdout);

	pthread_exit(NULL);
}


int
main (int argc, char *argv[])
{
	pthread_t thread[N_THREADS];
	int result;
  long int t;
	char err_msg[LEN];

	//	int pthread_spin_init(pthread_spinlock_t *lock, int pshared);
	pthread_spin_init(&_slock, PTHREAD_PROCESS_PRIVATE);       // inicializaçãoo da estrutura spin

	// cria threads
	for(t=0; t < N_THREADS; t++) {
		// printf("Criando thread %d\n", t);
		if((result=pthread_create(&thread[t], NULL, soma, (void *)t))) {
			strerror_r(result,err_msg,LEN);
			printf("Erro criando thread: %s\n",err_msg);
			exit(0);
		}
	}

	// espera threads conclu?rem
	for(t=0; t < N_THREADS; t++) {
		if((result=pthread_join(thread[t], NULL))) {
			strerror_r(result,err_msg,LEN);
			printf("Erro em pthread_join: %s\n",err_msg);
			exit(0);
		}
	}

	printf("Soma final: %d\n",_result);

	// libera spin
	pthread_spin_destroy (&_slock);

	return 0;
}

Overwriting spin.c


In [ ]:
! if [ ! spin -nt spin.c ]; then gcc -Wall spin.c -o spin -pthread; fi
! ./spin

@helio: to do...

https://man.openbsd.org/rwlock.9

https://man.openbsd.org/mutex.9

https://man7.org/linux/man-pages/man2/futex.2.html